In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.8 MB/s eta 0:00:00


In [ ]:
!ls
!pip install arabic_reshaper

In [ ]:

# Function to obtain the BERT representation for a given text
def get_bert_embedding(text):
    input_ids = torch.tensor(tokenizer.encode(text, return_tensors='pt')).unsqueeze(0)
    with torch.no_grad():
        output = model(input_ids)
    return output[0][0].mean(dim=0).numpy()

# Read the transcriptions from the text files and obtain their BERT embeddings
text_files = ["1.txt", "2.txt",  "3.txt",  "4.txt",  "5.txt",  "6.txt",  "7.txt",  "8.txt",  "9.txt",  "10.txt"]
for file in text_files:
    with open(file, 'r') as f:
        text = f.read()
    embedding = get_bert_embedding(text)
    # Save the embedding to a new file
    np.save(f"{file}.npy", embedding)
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import os
import torch.nn.functional as F
import re
# Load the Arabert model and tokenizer aubmindlab/bert-large-arabertv2
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabert")
model.config.max_position_embeddings=1024
# Path to the directory containing the text files
output_dir = './emb/'

# Path to the output TSV file
output_tsv_file = 'word_embeddings.tsv'

 
# Open the output TSV file for writing
# Loop through each file in the directory


def clean_text(text):
    # Remove extra spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()    
    return text

for filename in range(0, 60):

        # Read the contents of the file
        with open('data/'+str(filename)+'.txt', 'r') as f_in:
            text1 = f_in.read()
        text = clean_text(text1)
            
        # Tokenize the text into individual words
        input_ids = tokenizer.encode(text, return_tensors='pt')
        print(input_ids)
        input_id_chunks = input_ids.split(512,  dim=1)
        input_id_chunks = list(input_id_chunks)
        last_chunk_size = input_id_chunks[-1].size(1)
        if last_chunk_size < 512:
           padding_size = (0, 512 - last_chunk_size)
           input_id_chunks[-1] = F.pad(input_id_chunks[-1], padding_size, mode='constant', value=0)
        input_id_chunks = tuple(input_id_chunks)
        # Tokenize the text into individual words
        #input_id1s = tokenizer.encode(text1, return_tensors='pt')
    
        #input_id1_chunks = input_id1s.split(512,  dim=1)
        text_word_level=''
        with torch.no_grad():
            outputs = []
            # Pass the input tokens through the model
            for input in input_id_chunks:
              output = model(input)
              outputs.append(output[0])
            # Concatenate the outputs along the batch dimension
            print(outputs)
            last_hidden_states = torch.cat(outputs, dim=0)
            print(last_hidden_states.shape)
            print(last_hidden_states)
            n = last_hidden_states.shape[0]
            last_hidden_states = last_hidden_states.reshape(n * 512, 768)
            # Get the embeddings for each word in the input text
            word_embeddings = last_hidden_states 
            print(last_hidden_states.shape)
            print(last_hidden_states)
            print(word_embeddings.shape)
            with open(os.path.join(output_dir, f'{filename}.tsv'), 'w') as file:
               for i, word in enumerate(tokenizer.tokenize(text)):
                  text_word_level=text_word_level+'\n'+word
                  embedding = word_embeddings[i].numpy()
                  embedding_str = '\t'.join([str(x) for x in embedding])
                  file.write(f"{embedding_str}\n")
            with open(os.path.join(output_dir,f'{filename}.txt'),'w') as file:
                 file.write(f"{text_word_level}\n")   
print(f"Done. Word embeddings saved to {output_tsv_file}")

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (1406 > 512). Running this sequence through th

tensor([[29756,  5066,  8215,  ...,   648,   980, 29758]])
[tensor([[[-0.3848,  0.2553, -0.5977,  ..., -0.1821, -0.2831,  0.0091],
         [-0.7320, -0.2111, -0.3234,  ...,  0.1241, -0.0459, -0.0910],
         [-0.0705,  0.0298,  0.2991,  ...,  0.0594,  0.3374, -0.7536],
         ...,
         [-0.8211,  0.6623,  0.5571,  ..., -0.2401,  0.0219, -0.0518],
         [-0.3390,  0.0325,  0.4043,  ..., -0.6325, -0.3227,  0.1936],
         [-0.8686, -0.4973, -0.6481,  ..., -0.4772,  0.3653, -0.3027]]]), tensor([[[-0.3582,  0.4134, -0.6677,  ..., -0.2812, -0.1611, -0.1255],
         [ 0.1824,  0.2654,  0.4739,  ...,  0.0996, -0.5157, -0.3590],
         [ 0.6068,  0.2650,  0.9794,  ..., -0.0462, -0.4988, -1.1384],
         ...,
         [ 0.0497,  0.6623,  0.4828,  ...,  0.0080, -0.0339, -1.2060],
         [-0.2955,  0.0907,  0.0559,  ..., -0.1295, -0.1044, -0.6556],
         [-0.5734,  0.1512, -0.0082,  ..., -1.0392, -0.2882, -0.4135]]]), tensor([[[-2.0711e-01,  1.0114e+00,  4.4075e-01,  ...,

In [ ]:
!pip install conda